In [4]:
import pandas as pd
from pathlib2 import Path as pt
import os
from loguru import logger

metric = ['detect_icon_correct', 'detect_text_correct', 'detect_button_correct',
       'gold_text_number', 'gold_icon_number', 'gold_button_number',
       'predict_button_number', 'detect_icon_recall', 'detect_text_recall',
       'detect_button_recall', 'detect_button_precision',
       'classification_icon_correct', 'classification_text_correct',
       'classification_button_correct', 'predict_icon_number',
       'predict_text_number', 'classification_icon_precision',
       'classification_text_precision', 'classification_button_precision',
       'classification_icon_recall', 'classification_text_recall',
       'classification_button_recall', 'ocr_correct', 'predict_ocr_number',
       'gold_ocr_number', 'ocr_precision', 'ocr_recall']
metric2id={}
for k,v in enumerate(metric):
    metric2id[v] = k

In [5]:
curdir = r'D:\sa_data\exps\aware_softnms'
suffix = '20221117'
p = pt(curdir)
recall_w ,precision_w =0.8,0.2
new_df = pd.DataFrame()
f1_df = pd.DataFrame()
for i in p.iterdir():
    if suffix in i.name:
        logger.info(i.name[-8:])
        csv_parent_path = (i /"Element_grabbing"/"grabbing_evaluation")
        csv_dir = str(csv_parent_path)
        name = [str(x) for x in os.listdir(csv_dir) if x[-3:]=='csv']# if 'csv' in str(x)]
        if(len(name) == 0):
            continue
        csvpath=(csv_parent_path / name[0])
        try:
            df= pd.read_csv(str(csvpath), index_col=0)
            # arr = df.values
            # del arr
            stat_col = df.iloc[:,-1]
            # logger.info(stat_col.columns)
            col_name = f'{i.name[25:]}'
            stat_col.rename(col_name,inplace=True)
            # stat_col.iloc[:,0]  = str(stat_col.iloc[:,0]) + f'_{i.name[-4:]}'
            new_df = pd.concat([new_df,stat_col], axis=1)
            f11 = df.iloc[metric2id['classification_button_recall'],-1]* recall_w +  df.iloc[metric2id['classification_button_precision'],-1]* precision_w
            f12 = df.iloc[metric2id['detect_button_recall'],-1]* recall_w +  df.iloc[metric2id['detect_button_precision'],-1]* precision_w
            f1_col = pd.Series([f11,f12])
            f1_df = pd.concat([f1_df, f1_col],axis=1)
            # logger.info( f'classification f1:{f11}' )
            # logger.info( f'detect         f2:{f12}' )
        except :
            logger.info(1111)
            continue
new_df.to_csv((p /'diff_nms-iou_aware_conf_0_15.csv'))

# f1_df.to_csv('diff_conf_comprehensive.csv')

2022-11-18 10:49:19.836 | INFO     | __main__:<module>:9 - 15_nms30
2022-11-18 10:49:19.842 | INFO     | __main__:<module>:9 - 15_nms40
2022-11-18 10:49:19.848 | INFO     | __main__:<module>:9 - 15_nms50
2022-11-18 10:49:19.860 | INFO     | __main__:<module>:9 - 15_nms60
2022-11-18 10:49:19.871 | INFO     | __main__:<module>:9 - 15_nms70


In [6]:
if isinstance([1] , list):
    print(1)
else:
    print(2)

1


In [7]:
# a= False
# if a:
#     print(1)
# else:
#     print(2)

import torch
a = torch.randn(2, 10)
print(f'a: {a}')
print(f'第四列: {a[...,4].shape}')
print(f'第四列之后: {a[...,4:].shape}')
class_conf, class_pred = torch.max(a[:,6:], 1, keepdim=True) 
print(f'softmax之后:{class_conf.shape}' )
print(f'class_conf val:{class_conf}' )
print(f'class_pred:{class_pred}')
print(class_conf.squeeze())#拉成一维
obj_conf = a[:,5].view(2,1)
label_conf = obj_conf* class_conf
print(f'label_conf:{label_conf}')
detections = torch.cat((a[:, :4], label_conf, class_pred.float()), 1)
print(detections)

a: tensor([[ 0.0974, -1.8731, -1.0122, -0.8655,  0.1691, -1.3724, -0.4229,  1.2071,
         -1.8039,  0.6355],
        [ 0.3191, -0.5842, -0.2021, -1.0984,  0.6804,  0.8931, -1.2210, -1.1216,
         -0.3647, -0.5966]])
第四列: torch.Size([2])
第四列之后: torch.Size([2, 6])
softmax之后:torch.Size([2, 1])
class_conf val:tensor([[ 1.2071],
        [-0.3647]])
class_pred:tensor([[1],
        [2]])
tensor([ 1.2071, -0.3647])
label_conf:tensor([[-1.6567],
        [-0.3257]])
tensor([[ 0.0974, -1.8731, -1.0122, -0.8655, -1.6567,  1.0000],
        [ 0.3191, -0.5842, -0.2021, -1.0984, -0.3257,  2.0000]])


In [8]:
def np_1row_to_1cols(arr):
    return arr.reshape((len(arr), 1))

In [9]:
num_classes =3
aa = torch.randn(1, 10, 8)
for i , image_pred in enumerate(aa):
    print(i)
    class_conf, class_pred = torch.max(image_pred[..., 5:5+ num_classes], 1, keepdim=True) 
print(aa)
aa_r =torch.squeeze(aa,0)
aa_np = aa.numpy()
n_prediction = aa_r.numpy()

x1 = n_prediction[:,0] - n_prediction[:,2]/2
y1= n_prediction[:,1] - n_prediction[:,3]/2
x2 = n_prediction[:,0] + n_prediction[:,2]/2
y2 = n_prediction[:,1] + n_prediction[:,3]/2

x1 = np_1row_to_1cols(x1)
x2 = np_1row_to_1cols(x2)
y1 = np_1row_to_1cols(y1)
y2 = np_1row_to_1cols(y2)
front_obj = np_1row_to_1cols(n_prediction[:,4])
import numpy as np
bbox = np.concatenate([x1,y1,x2,y2], axis=1)
# scores = np.max(n_prediction[:,5:],axis=0)
class_conf = class_conf.numpy()
class_pred = class_pred.numpy()
dets = np.concatenate([x1,y1,x2,y2,front_obj*class_conf], axis=1)
yolox_7dets = np.concatenate([x1,y1,x2,y2,front_obj,class_conf,class_pred], axis=1)
# print(f'front:{front_obj}* class_conf{class_conf} = {front_obj* class_conf}')

0
tensor([[[ 1.1480,  2.1804, -0.2305, -2.2331, -0.4483,  0.6190, -0.1804,
           0.8403],
         [-1.7459,  0.2876,  0.1711, -1.5251,  0.6929, -0.4092, -0.5314,
           0.2585],
         [ 1.2563,  0.1614,  0.3163, -1.0393, -0.4135,  0.4851, -1.9443,
          -2.3526],
         [ 0.9112, -0.7693, -1.3225,  0.1931, -0.8080,  1.9815,  0.6644,
          -0.0128],
         [ 1.6553, -0.1737, -0.3121, -1.6465, -0.9131,  0.7224,  0.1809,
          -1.7943],
         [ 1.2200,  1.1082, -0.4577,  2.2572, -1.5023, -0.7436,  1.1598,
           0.4430],
         [ 1.1165,  1.5345, -0.1262,  0.7666, -0.0045, -0.8615,  1.0758,
          -0.2922],
         [-1.8319,  0.1580,  0.1963, -0.1688, -0.2320,  0.0913,  0.1511,
          -0.6363],
         [ 0.5766, -0.5393,  0.8972, -0.3446, -0.5989, -0.0133, -0.4370,
           0.4522],
         [ 1.4481,  0.4614,  0.5390,  0.1819,  0.1651, -0.4267, -0.5884,
          -1.5828]]])


In [10]:
def soft_nms_1(dets, sigma=0.5, nms_thr=0.5, method=2, conf_thr=0.1):
    box_nums = len(dets)   # box的个数
    for i in range(box_nums):
        tmpx1, tmpy1, tmpx2, tmpy2, ts = dets[i, 0], dets[i, 1], dets[i, 2], dets[i, 3], dets[i, 4]
        max_pos = i
        max_scores = ts

        # get max box
        pos = i+1
        while pos < box_nums:
            if max_scores < dets[pos, 4]:
                max_scores = dets[pos, 4]
                max_pos = pos
            pos += 1

        # add max box as a detection
        dets[i, :] = dets[max_pos, :]

        # swap ith box with position of max box
        dets[max_pos, 0] = tmpx1
        dets[max_pos, 1] = tmpy1
        dets[max_pos, 2] = tmpx2
        dets[max_pos, 3] = tmpy2
        dets[max_pos, 4] = ts

        # 将置信度最高的 box 赋给临时变量
        tmpx1, tmpy1, tmpx2, tmpy2, ts = dets[i, 0], dets[i, 1], dets[i, 2], dets[i, 3], dets[i, 4]

        pos = i+1
        # NMS iterations, note that box_nums changes if detection boxes fall below threshold
        while pos < box_nums:
            x1, y1, x2, y2 = dets[pos, 0], dets[pos, 1], dets[pos, 2], dets[pos, 3]

            area = (x2 - x1 + 1)*(y2 - y1 + 1)

            iw = (min(tmpx2, x2) - max(tmpx1, x1) + 1)
            ih = (min(tmpy2, y2) - max(tmpy1, y1) + 1)
            if iw > 0 and ih > 0:
                overlaps = iw * ih
                ious = overlaps / ((tmpx2 - tmpx1 + 1) * (tmpy2 - tmpy1 + 1) + area - overlaps)

                if method == 1:    # 线性
                    if ious > nms_thr:
                        weight = 1 - ious
                    else:
                        weight = 1
                elif method == 2:  # gaussian
                    weight = np.exp(-(ious**2) / sigma)
                else:              # original NMS
                    if ious > nms_thr:
                        weight = 0
                    else:
                        weight = 1

                # 1.赋予该box新的置信度
                dets[pos, 4] = weight * dets[pos, 4]

                # 2.如果box得分仍然低于阈值thresh，则通过与最后一个框交换来丢弃该框
                if dets[pos, 4] < conf_thr:
                    dets[pos, 0] = dets[box_nums-1, 0]
                    dets[pos, 1] = dets[box_nums-1, 1]
                    dets[pos, 2] = dets[box_nums-1, 2]
                    dets[pos, 3] = dets[box_nums-1, 3]
                    dets[pos, 4] = dets[box_nums-1, 4]

                    box_nums = box_nums-1
                    pos = pos-1
            pos += 1
    #拿出所有id
    keep = [i for i in range(box_nums)]
    # return dets[keep]
    return keep

def py_cpu_softnms(dets, sc, Nt=0.3, sigma=0.5, thresh=0.1, method=2):
    """
    py_cpu_softnms
    :param dets:   boexs 坐标矩阵 format [y1, x1, y2, x2]
    :param sc:     每个 boxes 对应的分数
    :param Nt:     iou 交叠门限
    :param sigma:  使用 gaussian 函数的方差
    :param thresh: 最后的分数门限
    :param method: 使用的方法
    :return:       留下的 boxes 的 index
    """

    # indexes concatenate boxes with the last column
    N = dets.shape[0]
    indexes = np.array([np.arange(N)])
    dets = np.concatenate((dets, indexes.T), axis=1)

    # the order of boxes coordinate is [y1,x1,y2,x2]
    y1 = dets[:, 0]
    x1 = dets[:, 1]
    y2 = dets[:, 2]
    x2 = dets[:, 3]
    scores = sc
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)

    for i in range(N):
        # intermediate parameters for later parameters exchange
        tBD = dets[i, :].copy()
        tscore = scores[i].copy()
        tarea = areas[i].copy()
        pos = i + 1

        #
        if i != N-1:
            maxscore = np.max(scores[pos:], axis=0)
            maxpos = np.argmax(scores[pos:], axis=0)
        else:
            maxscore = scores[-1]
            maxpos = 0
        if tscore < maxscore:
            dets[i, :] = dets[maxpos + i + 1, :]
            dets[maxpos + i + 1, :] = tBD
            tBD = dets[i, :]

            scores[i] = scores[maxpos + i + 1]
            scores[maxpos + i + 1] = tscore
            tscore = scores[i]

            areas[i] = areas[maxpos + i + 1]
            areas[maxpos + i + 1] = tarea
            tarea = areas[i]

        # IoU calculate
        xx1 = np.maximum(dets[i, 1], dets[pos:, 1])
        yy1 = np.maximum(dets[i, 0], dets[pos:, 0])
        xx2 = np.minimum(dets[i, 3], dets[pos:, 3])
        yy2 = np.minimum(dets[i, 2], dets[pos:, 2])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[pos:] - inter)

        # Three methods: 1.linear 2.gaussian 3.original NMS
        if method == 1:  # linear
            weight = np.ones(ovr.shape)
            weight[ovr > Nt] = weight[ovr > Nt] - ovr[ovr > Nt]
        elif method == 2:  # gaussian
            weight = np.exp(-(ovr * ovr) / sigma)
        else:  # original NMS
            weight = np.ones(ovr.shape)
            weight[ovr > Nt] = 0

        scores[pos:] = weight * scores[pos:]
    print(type(scores))
    # select the boxes and keep the corresponding indexes
    inds = dets[:, 4][scores > thresh]
    print(dets[:, 4][scores > thresh])
    print(inds)
    keep = inds.astype(int)

    return keep

In [11]:
a= [[1],[2]]
a= np.array(a)


In [12]:
print(a)
a.reshape((2,1))
print(a)

[[1]
 [2]]
[[1]
 [2]]


In [13]:
# aaaaa = soft_nms_1(dets=dets)
c = class_conf.reshape((1,-1))
id = py_cpu_softnms(bbox, c[0])

print(yolox_7dets.shape)
yolox_7dets = yolox_7dets[id]
print(yolox_7dets.shape)


<class 'numpy.ndarray'>
[3. 5. 0. 6. 4. 2. 8. 1. 7.]
[3. 5. 0. 6. 4. 2. 8. 1. 7.]
(10, 7)
(9, 7)
